In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf
from tensorflow import keras

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Tensorflow version:", tf.__version__)
print("Eager mode:", tf.executing_eagerly())
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Tensorflow version: 2.0.0
Eager mode: True
Hub version: 0.6.0
GPU is NOT AVAILABLE


In [19]:
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews",
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

In [8]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch[0]

<tf.Tensor: id=265, shape=(), dtype=string, numpy=b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by a 

In [9]:
train_labels_batch

<tf.Tensor: id=261, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [11]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=611, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [14]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add (tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [16]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 167ms/step - loss: 0.7775 - accuracy: 0.5316 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 4s 123ms/step - loss: 0.6651 - accuracy: 0.6097 - val_loss: 0.6395 - val_accuracy: 0.6397
Epoch 3/20
30/30 [==============================] - 4s 138ms/step - loss: 0.6210 - accuracy: 0.6611 - val_loss: 0.6047 - val_accuracy: 0.6777
Epoch 4/20
30/30 [==============================] - 4s 120ms/step - loss: 0.5883 - accuracy: 0.6935 - val_loss: 0.5753 - val_accuracy: 0.7043
Epoch 5/20
30/30 [==============================] - 4s 134ms/step - loss: 0.5506 - accuracy: 0.7231 - val_loss: 0.5439 - val_accuracy: 0.7330
Epoch 6/20
30/30 [==============================] - 4s 125ms/step - loss: 0.5168 - accuracy: 0.7555 - val_loss: 0.5106 - val_accuracy: 0.7571
Epoch 7/20
30/30 [==============================] - 5s 176ms/step - loss: 0.4787 - accuracy: 0.7845 - val_loss: 0.4772 - val_accuracy: 0.781

In [18]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("{}: {}".format(name, value))

49/49 - 2s - loss: 0.3105 - accuracy: 0.8664
loss: 0.3104758505918542
accuracy: 0.8664000034332275
